In [1]:
import importlib.util
def module_from_file(module_name, file_path): #The code that imports the file which originated the training with all the instructions
            spec = importlib.util.spec_from_file_location(module_name, file_path)
            module = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(module)
            return module

from importlib import import_module
#foo = import_module(fold_folder+'/Funs.py', package=None)


folder = './xforanalogs/NA24by48/Z8/yrs500/interT15fw20.1.20lrs4'
foo = module_from_file("foo", f'{folder}/Funs.py')

import random as rd  
from scipy.stats import norm
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.metrics import log_loss

tff = foo.tff # tensorflow routines 
ut = foo.ut # utilities
ln = foo.ln #Learn2_new.py
print("==Checking GPU==")
import tensorflow as tf
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

print("==Checking CUDA==")
tf.test.is_built_with_cuda()

#from tensorflow.keras.preprocessing.image import ImageDataGenerator
sys.path.insert(1, '../ERA')

print("==Reading data==")

year_permutation = np.load(f'{folder}/year_permutation.npy')

2022-06-30 07:43:27.785464: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
In this environment you cannot import Basemap
In this environment you cannot import Basemap


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
==Checking GPU==
==Checking CUDA==
==Reading data==


2022-06-30 07:43:31.286621: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-30 07:43:31.293164: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-30 07:43:31.529881: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-30 07:43:31.529937: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (apollo2048g): /proc/driver/nvidia/version does not exist
2022-06-30 07:43:31.529957: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices

In [2]:
run_vae_kwargs = ut.json2dict(f"{folder}/config.json")
T = ut.extract_nested(run_vae_kwargs, 'T')
if (ut.keys_exists(run_vae_kwargs, 'label_period_start') and ut.keys_exists(run_vae_kwargs, 'label_period_end')):
    label_period_start = ut.extract_nested(run_vae_kwargs, 'label_period_start')
    label_period_end = ut.extract_nested(run_vae_kwargs, 'label_period_end')
    time_start = ut.extract_nested(run_vae_kwargs, 'time_start')
    time_end = ut.extract_nested(run_vae_kwargs, 'time_end')

In [3]:
n_days = time_end-time_start-T+1

In [7]:
import pickle
fold = 0
open_file = open(f'{folder}/fold_{fold}/analogues.pkl', "rb")
analogues = pickle.load(open_file)
open_file.close()

analogues['ind_new_tr'][10]//n_days

array([[  0, 241,   0, ...,  62, 108, 102],
       [  0,  54, 408, ...,   5, 277, 269],
       [  0, 408,   0, ...,   2, 118, 101],
       ...,
       [449,  34, 330, ..., 436, 105, 243],
       [208, 211, 211, ..., 208,  41,  14],
       [211, 208, 211, ..., 346, 386,  67]])

In [8]:
analogues['ind_new_tr'][10]

array([[    0, 25317,     1, ...,  6593, 11410, 10742],
       [    1,  5699, 42840, ...,   574, 29142, 28343],
       [    2, 42841,     3, ...,   283, 12414, 10665],
       ...,
       [47246,  3621, 34707, ..., 45849, 11090, 25576],
       [21920, 22243, 22242, ..., 21925,  4376,  1553],
       [22243, 21920, 22244, ..., 36384, 40564,  7108]])

In [9]:
import numpy as np
selfanalogues = (analogues['ind_new_tr'][1000]//n_days)[:,0][np.newaxis].T
print(selfanalogues.shape)
sameyear = selfanalogues*np.ones((selfanalogues.shape[0],1000))==analogues['ind_new_tr'][1000]//n_days
np.mean(np.sum(sameyear,1)), np.std(np.sum(sameyear,1))

(47250, 1)


(11.846306878306878, 7.023720777239038)

In [12]:
analogues['ind_new_tr'][10][:9,:5]

array([[    0, 25317,     1, 32025,  5702],
       [    1,  5699, 42840,  5701,  5700],
       [    2, 42841,     3, 42544,  5699],
       [    3,     4,     2,  5367, 32558],
       [    4,     3,     5,  5367,  5368],
       [    5,     4,     6,  5368,     3],
       [    6,     5,     7, 47155, 47154],
       [    7, 47155, 26782,     6, 32563],
       [    8, 27245, 27246, 16208, 42014]])